# LSTM Forecasting

In [1]:
# --- Repo path bootstrap so imports like `from price_forecast...` work ---
import os, sys
REPO_ROOT = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
SRC_ROOT = os.path.join(REPO_ROOT, "src")
if SRC_ROOT not in sys.path:
    sys.path.insert(0, SRC_ROOT)

# Optional, for iterative dev:

data_dir = os.path.join(REPO_ROOT, "data")  # where data is stored

In [2]:
SRC_ROOT

'c:\\Users\\HarrisDeralas\\Downloads\\panagiotis_files\\price_forecast\\src'

In [3]:
data_dir

'c:\\Users\\HarrisDeralas\\Downloads\\panagiotis_files\\price_forecast\\data'

## Build the dataset

In [4]:
import pandas as pd
import numpy as np

from price_forecast.config import (
    TimezoneConfig, Naming, DAMConfig, WeatherConfig, WeatherSite
)
from price_forecast.pipeline.dataset_builder import DatasetBuilder
from price_forecast.utils.merge import TimeSeriesMerger


In [5]:
from entsoe import EntsoePandasClient
client = EntsoePandasClient(api_key="7260b39e-b41a-4905-a402-4a65bd531f3f")

In [6]:
tz_cfg = TimezoneConfig()
naming = Naming(dt_local="datetime_local", dam_price="dam_price_eur_mwh")
dam_cfg = DAMConfig(base_dir="../data")
sites = [
    WeatherSite("athens", 37.98, 23.72),
    WeatherSite("thessaloniki", 40.64, 22.94),
    WeatherSite("heraklion", 35.34, 25.13),
]
wx_cfg = WeatherConfig()

In [7]:
builder = DatasetBuilder(
    tz_cfg=tz_cfg,
    naming=naming,
    dam_cfg=dam_cfg,
    weather_sites=sites,
    weather_cfg=wx_cfg,
    entsoe_client=client,
    entsoe_prefix="entsoe_",
)

In [8]:
df  = builder.create_dataset(start_date="2020-11-01", end_date="2023-12-31", add_prev_day=True)

Finished building DAM data.
Finished building ENTSO-E data.
Finished building Weather data.
Finished merging all data on UTC axis.
Converted merged data back to local timezone.
Added previous-day DAM price feature.


## Visualize the DAM series

In [13]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df["datetime_local"],
        y=df["dam_price_eur_mwh"],
        name="DAM Price",
        mode="lines"
    )
)

fig.add_trace(
    go.Scatter(
        x=df["datetime_local"],
        y=df["previous_day_dam"],
        name="Previous Day DAM Price",
        mode="lines"
    )
)

fig.update_layout(
    title="Day-Ahead Market Prices",
    xaxis_title="Date",
    yaxis_title="Price (EUR/MWh)",
    hovermode="x unified",
    xaxis=dict(rangeslider=dict(visible=True), type="date"),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


## Generate dataset for LSTM ingestion

In [18]:
from price_forecast.config import DatasetCfg

cfgd = DatasetCfg(n_lookback_days=30,test_size=0.2,put_main_first = True)

In [19]:
from price_forecast.datasets.daily_sequence import DailySequenceDataset

builder = DailySequenceDataset(df, cfgd)
data = builder.build(return_dfs=True)

# scaled for training
X_train, y_train = data["X_train"], data["y_train"]

# unscaled for metrics/plots/inverse checks
X_train_raw, y_train_raw = data["X_train_raw"], data["y_train_raw"]


# tidy inspection (raw vs scaled)
X_train_df_raw = data["X_train_df_raw"]
X_train_df_scaled = data["X_train_df"]
y_train_df_raw = data["y_train_df_raw"]
X_test_df_raw = data["X_test_df_raw"]
y_test_df_raw = data["y_test_df_raw"]

c:\Users\HarrisDeralas\Downloads\panagiotis_files\price_forecast\src\price_forecast\datasets\daily_sequence.py:237: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  if not np.all(diffs.view("i8") == pd.Timedelta(hours=1).value):


In [17]:
cfgd = DatasetCfg(
    datetime_col="datetime_local",
    target_col="dam_price_eur_mwh",
    n_lookback_days=60,
    test_size=0.2,
    main_series="previous_day_dam",
    put_main_first=True,
)

builder = DailySequenceDataset(df, cfgd)
data = builder.build(return_dfs=True)

# scaled for training
X_train, y_train = data["X_train"], data["y_train"]

# unscaled for metrics/plots/inverse checks
X_train_raw, y_train_raw = data["X_train_raw"], data["y_train_raw"]

# tidy inspection (raw vs scaled)
X_train_df_raw = data["X_train_df_raw"]
X_train_df_scaled = data["X_train_df"]
y_train_df_raw = data["y_train_df_raw"]
X_test_df_raw = data["X_test_df_raw"]
y_test_df_raw = data["y_test_df_raw"]


# scaled arrays for training
X_train, X_test = data["X_train"], data["X_test"]
y_train, y_test = data["y_train"], data["y_test"]



c:\Users\HarrisDeralas\Downloads\panagiotis_files\price_forecast\src\price_forecast\datasets\daily_sequence.py:139: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.



## Train the LSTM(s)

In [18]:
# X_train: (batch, input_steps, n_features)
# y_train: (batch, output_steps, output_dim)
from price_forecast.models.lstm_seq2seq import LSTMSeq2SeqModel
model = LSTMSeq2SeqModel.from_training_data(
    X_train, y_train,
    lstm_units=15*8,
    dropout=0.2,
)

# Optional explicit tail validation split
split = int(0.9 * X_train.shape[0])
history = model.fit(
    X_train=X_train[:split], y_train=y_train[:split],
    X_val=X_train[split:],   y_val=y_train[split:],
    epochs=60, batch_size=64, patience=10, reduce_lr_patience=5,
    checkpoint_path="best_lstm.keras",
    verbose=1,
)

# Predict, invert scaling with your builder, evaluate, and persist
y_pred_scaled = model.predict(X_test)
y_pred = builder.inverse_transform_target(y_pred_scaled)
y_true = builder.inverse_transform_target(y_test)
print(LSTMSeq2SeqModel.evaluate(y_true, y_pred))





c:\Users\HarrisDeralas\anaconda3\envs\price-forecast\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "lstm_seq2seq_day_ahead"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_lstm (LSTM)             │ (None, 120)            │        71,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_dropout (Dropout)       │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_output_steps             │ (None, 24, 120)        │             0 │
│ (RepeatVector)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm (LSTM)             │ (None, 24, 120)        │       115,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_dropout (Dropout)       │ (None, 24, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hourly_output (TimeDistributed) │ (None, 24, 1)          │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,321 (731.72 KB)

 Trainable params: 187,321 (731.72 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 1.2069 - mae: 0.9508
Epoch 1: val_loss improved from None to 0.18644, saving model to best_lstm.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - loss: 1.5690 - mae: 1.0026 - val_loss: 0.1864 - val_mae: 0.3560 - learning_rate: 0.0010
Epoch 2/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.4189 - mae: 0.5557
Epoch 2: val_loss did not improve from 0.18644
13/13 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.4424 - mae: 0.5240 - val_loss: 0.1941 - val_mae: 0.3651 - learning_rate: 0.0010
Epoch 3/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.3110 - mae: 0.4616
Epoch 3: val_loss improved from 0.18644 to 0.14745, saving model to best_lstm.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 0.3796 - mae: 0.4818 - val_loss: 0.1475 - val_mae: 0.3053 - learning_rate: 0.0010
Epoch 4/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 0.2670 - mae: 0.3971
Epoch 4: val_loss did not improve from 0.14745
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - l

In [19]:
# --- Imports (adjust the import path to your package layout) ---
from price_forecast.models.lstm_seq2seq_plus import LSTMSeq2SeqPlus  # the new model
from price_forecast.models.lstm_seq2seq import LSTMSeq2SeqModel      # old model (for evaluate reuse)

# X_train: (batch, input_steps, n_features)
# y_train: (batch, output_steps, output_dim)

# 1) Build the model by inferring shapes from data, with all hyperparams explicit
model_plus = LSTMSeq2SeqPlus.from_training_data(
    X_train=X_train,
    y_train=y_train,
    # --- Architecture ---
    enc_units=(128, 64),        # stacked encoder LSTMs; try (256,128) or (64,32)
    dec_units=128,              # decoder LSTM size; try 64/256
    bidirectional=True,         # use BiLSTM in encoder
    attention=True,             # AdditiveAttention over encoder sequence
    dropout=0.2,                # dropout between LSTM timesteps (Keras 'dropout')
    recurrent_dropout=0.0,      # use sparingly; slows training if >0
    layernorm=True,             # LayerNormalization after enc/dec
    residual_last24=True,       # concat projected last-24 inputs into the head
    # --- Training config (compile) ---
    optimizer="adam",
    loss="mse",
    metrics=("mae",),           # can add more, e.g. ("mae","mape")
    # --- Reproducibility ---
    seed=42,
)

# 2) Train (explicit tail validation split to keep temporal order)
split = int(0.9 * X_train.shape[0])
history_plus = model_plus.fit(
    X_train=X_train[:split],
    y_train=y_train[:split],
    X_val=X_train[split:],
    y_val=y_train[split:],
    epochs=80,                 # a bit higher by default for the larger model
    batch_size=64,
    patience=10,
    min_delta=1e-4,
    reduce_lr_patience=5,
    checkpoint_path="best_lstm_plus.keras",  # saves the full model
    verbose=1,
    shuffle=False,             # keep temporal ordering
)

# 3) Predict (scaled), inverse-transform, evaluate, and persist
y_pred_scaled_plus = model_plus.predict(X_test, batch_size=256, verbose=0)
y_pred_plus = builder.inverse_transform_target(y_pred_scaled_plus)
y_true = builder.inverse_transform_target(y_test)

print("PLUS metrics:", LSTMSeq2SeqModel.evaluate(y_true, y_pred_plus))  # reuse the same evaluate helper

# 4) Save & (optionally) restore
model_plus.save("final_lstm_plus.keras")
restored_plus = LSTMSeq2SeqPlus.load("best_lstm_plus.keras")


Model: "lstm_seq2seq_plus"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 1440, 28)  │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidir_enc_1         │ [(None, 1440,     │    160,768 │ inputs[0][0]      │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_ln_1            │ (None, 1440, 256) │        512 │ bidir_enc_1[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidir_enc_2         │ [(None, 1440,     │    164,352 │ enc_ln_1[0][0]    │
│ (Bidirectional)     │ 128), (None, 64), │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_ln_2            │ (None, 1440, 128) │        256 │ bidir_enc_2[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_avg_pool        │ (None, 128)       │          0 │ enc_ln_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_2_h_concat      │ (None, 128)       │          0 │ bidir_enc_2[0][1… │
│ (Concatenate)       │                   │            │ bidir_enc_2[0][3] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_2_c_concat      │ (None, 128)       │          0 │ bidir_enc_2[0][2… │
│ (Concatenate)       │                   │            │ bidir_enc_2[0][4] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_output_steps │ (None, 24, 128)   │          0 │ enc_avg_pool[0][… │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ init_h (Dense)      │ (None, 128)       │     16,512 │ enc_2_h_concat[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ init_c (Dense)      │ (None, 128)       │     16,512 │ enc_2_c_concat[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm (LSTM)     │ [(None, 24, 128), │    131,584 │ repeat_output_st… │
│                     │ (None, 128),      │            │ init_h[0][0],     │
│                     │ (None, 128)]      │            │ init_c[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_ln              │ (None, 24, 128)   │        256 │ dec_lstm[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention  │ (None, 24, 128)   │        128 │ dec_ln[0][0],     │
│ (AdditiveAttention) │                   │            │ enc_ln_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last24_slice        │ (None, 24, 28)    │          0 │ inputs[0][0]      │
│ (Cropping1D)        │                   │            │                 

 Total params: 526,417 (2.01 MB)

 Trainable params: 526,417 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "lstm_seq2seq_plus"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer) │ (None, 1440, 28)  │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidir_enc_1         │ [(None, 1440,     │    160,768 │ inputs[0][0]      │
│ (Bidirectional)     │ 256), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128), (None,      │            │                   │
│                     │ 128)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_ln_1            │ (None, 1440, 256) │        512 │ bidir_enc_1[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidir_enc_2         │ [(None, 1440,     │    164,352 │ enc_ln_1[0][0]    │
│ (Bidirectional)     │ 128), (None, 64), │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_ln_2            │ (None, 1440, 128) │        256 │ bidir_enc_2[0][0] │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_avg_pool        │ (None, 128)       │          0 │ enc_ln_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_2_h_concat      │ (None, 128)       │          0 │ bidir_enc_2[0][1… │
│ (Concatenate)       │                   │            │ bidir_enc_2[0][3] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ enc_2_c_concat      │ (None, 128)       │          0 │ bidir_enc_2[0][2… │
│ (Concatenate)       │                   │            │ bidir_enc_2[0][4] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_output_steps │ (None, 24, 128)   │          0 │ enc_avg_pool[0][… │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ init_h (Dense)      │ (None, 128)       │     16,512 │ enc_2_h_concat[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ init_c (Dense)      │ (None, 128)       │     16,512 │ enc_2_c_concat[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_lstm (LSTM)     │ [(None, 24, 128), │    131,584 │ repeat_output_st… │
│                     │ (None, 128),      │            │ init_h[0][0],     │
│                     │ (None, 128)]      │            │ init_c[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dec_ln              │ (None, 24, 128)   │        256 │ dec_lstm[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention  │ (None, 24, 128)   │        128 │ dec_ln[0][0],     │
│ (AdditiveAttention) │                   │            │ enc_ln_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ last24_slice        │ (None, 24, 28)    │          0 │ inputs[0][0]      │
│ (Cropping1D)        │                   │            │                 

 Total params: 526,417 (2.01 MB)

 Trainable params: 526,417 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

## Naive predictions (benchmark) [same as previous day]

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

test_start_date = y_test_df_raw['datetime'].min()
test_df = df[df['datetime_local'] >= test_start_date].copy()
naive_mae = mean_absolute_error(test_df['dam_price_eur_mwh'], test_df['previous_day_dam'])
naive_mse = mean_squared_error(test_df['dam_price_eur_mwh'], test_df['previous_day_dam'])
print("Naive (previous day) baseline:")
print("MSE:", naive_mse, "MAE:", naive_mae)

NameError: name 'y_test_df_raw' is not defined

## Test with relative features

In [9]:
datetime_col = "datetime_local"
target_col   = "dam_price_eur_mwh"
main_series  = "previous_day_dam"

In [10]:
from sklearn.preprocessing import StandardScaler
from price_forecast.config import DatasetCfg
from price_forecast.config import SafeStandardScaler

cfg = DatasetCfg(
    datetime_col=datetime_col,
    target_col=target_col,
    main_series=main_series,        # used for feature ordering
    put_main_first=True,            # keep previous_day_dam as first feature
    n_lookback_days=30,              # encoder lookback (=> input_steps = 7*24)
    test_size=0.2,                  # tail split by whole days
    scale_features=True,
    feature_scaler=SafeStandardScaler(),
    scale_target=True,
    target_scaler=StandardScaler())

In [11]:
from price_forecast.datasets.daily_sequence import DailySequenceDataset
builder = DailySequenceDataset(
    df, cfg,
    target_as_relative=True,
    relative_feature_cols=[main_series],  # turn previous_day_dam into rel-diffs
    epsilon=1e-6,
    fill_value=0.0,
)

c:\Users\HarrisDeralas\Downloads\panagiotis_files\price_forecast\src\price_forecast\datasets\daily_sequence.py:248: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  if not np.all(diffs.view("i8") == pd.Timedelta(hours=1).value):


In [12]:
out = builder.build(return_dfs=True)

X_train, X_test = out["X_train"], out["X_test"]        # (S, N*24, F)
y_train, y_test = out["y_train"], out["y_test"]        # (S, 24, 1)


# tidy inspection (raw vs scaled)
X_train_df_raw = out["X_train_df_raw"]
X_train_df_scaled = out["X_train_df"]
y_train_df_raw = out["y_train_df_raw"]
X_test_df_raw = out["X_test_df_raw"]
y_test_df_raw = out["y_test_df_raw"]

# For relative targets, we’ll need these to invert predictions back to absolute:
y_train_bases = out.get("y_train_bases", None)
y_test_bases  = out.get("y_test_bases", None)

print(
    f"X_train: {X_train.shape}, y_train: {y_train.shape} | "
    f"X_test: {X_test.shape}, y_test: {y_test.shape}"
)


X_train: (901, 720, 28), y_train: (901, 24, 1) | X_test: (225, 720, 28), y_test: (225, 24, 1)


In [13]:
from price_forecast.models.lstm_seq2seq import LSTMSeq2SeqModel

model = LSTMSeq2SeqModel.from_training_data(
    X_train, y_train,
    lstm_units=15 * 8,      # your earlier choice
    dropout=0.2,
    recurrent_dropout=0.0,
    optimizer="adam",
    loss="mse",
    metrics=("mae",),
    seed=42,
)

# Tail validation split is handled inside .fit() if you don't pass X_val/y_val
history = model.fit(
    X_train=X_train, y_train=y_train,
    epochs=60,
    batch_size=64,
    patience=10,
    reduce_lr_patience=5,
    checkpoint_path="best_lstm.keras",   # saves the full model
    verbose=1,
    shuffle=False,                       # keep temporal ordering
)



c:\Users\HarrisDeralas\anaconda3\envs\price-forecast\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "lstm_seq2seq_day_ahead"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_lstm (LSTM)             │ (None, 120)            │        71,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_dropout (Dropout)       │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_output_steps             │ (None, 24, 120)        │             0 │
│ (RepeatVector)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm (LSTM)             │ (None, 24, 120)        │       115,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_dropout (Dropout)       │ (None, 24, 120)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hourly_output (TimeDistributed) │ (None, 24, 1)          │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 187,321 (731.72 KB)

 Trainable params: 187,321 (731.72 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 902ms/step - loss: 0.4594 - mae: 0.3917
Epoch 1: val_loss improved from None to 0.51244, saving model to best_lstm.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 1.0847 - mae: 0.6096 - val_loss: 0.5124 - val_mae: 0.4709 - learning_rate: 0.0010
Epoch 2/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 991ms/step - loss: 0.4720 - mae: 0.4131
Epoch 2: val_loss improved from 0.51244 to 0.50472, saving model to best_lstm.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 1.0916 - mae: 0.6274 - val_loss: 0.5047 - val_mae: 0.4588 - learning_rate: 0.0010
Epoch 3/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - loss: 0.4340 - mae: 0.3610
Epoch 3: val_loss improved from 0.50472 to 0.50100, saving model to best_lstm.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 933ms/step - loss: 1.0575 - mae: 0.5874 - val_loss: 0.5010 - val_mae: 0.4581 - learning_rate: 0.0010
Epoch 4/60
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 833ms/step - loss: 0.4287 - mae: 0.3568
Epoch 4: val_loss did not improve fro

In [14]:
y_pred_test_scaled = model.predict(X_test, batch_size=256, verbose=0)

In [15]:
# The dataset helper handles both scaling-only and relative+scaling modes.
if y_test_bases is not None:
    # Relative mode: provide per-sample bases
    y_pred_test_abs = builder.inverse_transform_target(y_pred_test_scaled, bases=y_test_bases)
    y_true_test_abs = builder.inverse_transform_target(y_test,             bases=y_test_bases)
else:
    # Absolute mode (no relative deltas): inverse scaling only (or no-op if disabled)
    y_pred_test_abs = builder.inverse_transform_target(y_pred_test_scaled)
    y_true_test_abs = builder.inverse_transform_target(y_test)

# -------------------------
# 7) Simple evaluation (MAE/MSE)
# -------------------------
metrics = LSTMSeq2SeqModel.evaluate(y_true_test_abs, y_pred_test_abs)
print("Test metrics (abs units):", metrics)

Test metrics (abs units): {'mse': 6.961839310183534e+65, 'mae': 1.217391687341922e+31}
